# Best Practices

In [2]:
using HomotopyContinuation, LinearAlgebra
set_default_compile(:none)

:none

## Try to avoid determinants and minors 

For many problems it is necessary to formulate the condition that vectors are linearly dependent or that a matrix is singular.

**Examples**
* [Computing the narrowest bottleneck of a variety](https://www.juliahomotopycontinuation.org/examples/sampling_bottlenecks/)
* Sampling the discriminant of a parameterized system


Algebraic geometers in partcular, often reach to determinants and minors to formulate such a condition.
However, for homotopy continuation approaches this can result in numerically challenging systems.

**Why?**

Homotopy continuation is a *geometric* solution method. We have a solution to a system, slightly perturb the coefficients of our system and try to find again a solution to the perturbed system. As a general rule of thumb, the larger the degree of the system the more difficult it is to find the solution to the perturbed system.

**Better approach**

1) Singular matrices

Consider the case that we want to encode that an $n \times n$ matrix $J$ is singular. Instead of requiring $\det(J) = 0$ we use an additional n-dimensional vector $v$ and write the (partial) system
$$
\begin{bmatrix} J \\ J * v \\ \sum_i v_i - 1 \end{bmatrix}
$$

### Example:

In [13]:
@var x y a
F = System([x^3 + a*x*y + 1, y^4 + 3 * x - 2], parameters = [a])
J = jacobian(F)

2×2 Matrix{Expression}:
 a*y + 3*x^2    a*x
           3  4*y^3

In [14]:
# BAD approach
G_bad = System([expressions(F); det(J)])

System of length 3
 3 variables: a, x, y

 1 + a*x*y + x^3
 -2 + 3*x + y^4
 -3*a*x + 4*y^3*(a*y + 3*x^2)

In [15]:
# GOOD approach
@var v[1:2]
G_good = System([expressions(F); J * v; sum(v) - 1])

System of length 5
 5 variables: a, v₁, v₂, x, y

 1 + a*x*y + x^3
 -2 + 3*x + y^4
 v₁*(a*y + 3*x^2) + a*x*v₂
 3*v₁ + 4*y^3*v₂
 -1 + v₁ + v₂

2) Linear dependent vectors

If you want to model that two vectors $v$ and $w$ are linearly dependent is usually much better to just impose the condition that there exists an $\lambda \neq 0$ such that $v = \lambda w$.
If you use parameter homotopies and start with a start solution where $v$ and $w$ are dependent, then imposing the condition $\lambda \neq 0$ is often unnecessary.

## Parameter homotopies

A powerful feature of parameter homotopies is the possibility to use the offline-online approach.

**Recall:**

Our previous discussion gives us the following approach for repeatedly solving a parameterized polynomial system $F(x;p)$ with parameter space $\mathbb{C}^m$.

1. Compute all isolated solutions $S_q$ for a general $q \in \mathbb{C}^m$ by some method (offline part - needs to be done only once)
2. For each parameter value $p \in \mathbb{C}^m$ of interest
    1. Construct the homotopy $H(x, t) = F(x; t q + (1-t) p)$
    2. For each $s \in S_q$ track the solution from $t=1$ to $t=0$. The result is $S_p$.

Offline part:

In [16]:
@var x a b
F = System([x^2 + a * x + b], parameters = [a, b])

x₁, x₂ = randn(ComplexF64, 2)
q = [-(x₁ + x₂), x₁ * x₂]
S_q = [[x₁], [x₂]]

2-element Vector{Vector{ComplexF64}}:
 [0.2663202712744034 + 0.6864186388794614im]
 [0.7779845825624262 + 0.5872047571010558im]

In [17]:
write_parameters("quad_start_params.txt", q)
write_solutions("quad_start_sols.txt", S_q)

Online part:

In [18]:
q = read_parameters("quad_start_params.txt")

2-element Vector{ComplexF64}:
  -1.0443048538368296 - 1.2736233959805172im
 -0.19587522503752267 + 0.6904076484364798im

In [19]:
S_q = read_solutions("quad_start_sols.txt")

2-element Vector{Vector{ComplexF64}}:
 [0.2663202712744034 + 0.6864186388794614im]
 [0.7779845825624262 + 0.5872047571010558im]

In [20]:
p = [-2.5, 3]
R_p = solve(F, S_q; start_parameters = q, target_parameters = p)

Result with 2 solutions
• 2 paths tracked
• 2 non-singular solutions (0 real)
• random_seed: 0x89af08c6


In [21]:
S_p = solutions(R_p)

2-element Vector{Vector{ComplexF64}}:
 [1.25 - 1.1989578808281798im]
 [1.25 + 1.19895788082818im]

<div style="border:2px gray; border-style:solid; padding: 0.5em"> This approach works best if the start parameters and target parameters are:
   <ul>
      <li>of the same scale, i.e., similar in magnitude</li>
      <li>from similar probability distributions</li>
  </ul>
</div>

<div style="border:2px gray; border-style:solid; padding: 0.5em"> Try to avoid "random" integers as e.g. produced by Macaulay2 since these result oft in numerically badly conditioned systems </div> 

## Sparse > dense or why elimination is not always good

If you are used to work with a symbolic computer algebra system, you are used to formulate your problem such that the problem has a minimal number of variables, e.g.,  by eliminating some of the variables.

The tradeoff there is often: less variables in exchange for a system with higher degrees and with more terms (i.e. "more dense").

For homotopy continuation methods this approach can actually be harmful since the resulting system can be subtanially more expensive to evaluate, numerically worse, and of higher degree than the original system.

#### Example:  Steiner's conic problem

Consider Steiner's conic problem of computing plane conics tangents to five given conics.

We can formulate the problem as

In [22]:
@var x[1:2]

f, u = dense_poly(x, 2; coeff_name = :u)

(u₆ + u₁*x₁^2 + u₃*x₂^2 + u₄*x₁ + u₅*x₂ + u₂*x₂*x₁, Variable[u₁, u₂, u₃, u₄, u₅, u₆])

In [23]:
# The coefficents of the five given conics
@var C[1:6, 1:5]
# The coordinates of the five points of tangency
@var P[1:2,1:5]

exprs = vcat(map(1:5) do i
    cᵢ = f(u => C[:,i])
    pᵢ = P[:,i]
    # The conic f and cᵢ are tangent at pᵢ
    # if f(pᵢ) = cᵢ(pᵢ) = det([∇f ∇Jᵢ])) = 0
    Jᵢ = differentiate([cᵢ, f], x)
    [
        f(x => pᵢ, u[6] => 1)
        cᵢ(x => pᵢ)
        det(Jᵢ)(x => pᵢ, u[6] => 1)
    ]
end...)

F = System(
        exprs;
        variables = [u[1:5]; vec(P)],
        parameters = vec(C)
)

System of length 15
 15 variables: u₁, u₂, u₃, u₄, u₅, P₁₋₁, P₂₋₁, P₁₋₂, P₂₋₂, P₁₋₃, P₂₋₃, P₁₋₄, P₂₋₄, P₁₋₅, P₂₋₅
 30 parameters: C₁₋₁, C₂₋₁, C₃₋₁, C₄₋₁, C₅₋₁, C₆₋₁, C₁₋₂, C₂₋₂, C₃₋₂, C₄₋₂, C₅₋₂, C₆₋₂, C₁₋₃, C₂₋₃, C₃₋₃, C₄₋₃, C₅₋₃, C₆₋₃, C₁₋₄, C₂₋₄, C₃₋₄, C₄₋₄, C₅₋₄, C₆₋₄, C₁₋₅, C₂₋₅, C₃₋₅, C₄₋₅, C₅₋₅, C₆₋₅

 1 + u₁*P₁₋₁^2 + u₃*P₂₋₁^2 + u₄*P₁₋₁ + u₅*P₂₋₁ + u₂*P₁₋₁*P₂₋₁
 C₆₋₁ + C₁₋₁*P₁₋₁^2 + C₃₋₁*P₂₋₁^2 + C₄₋₁*P₁₋₁ + C₅₋₁*P₂₋₁ + C₂₋₁*P₁₋₁*P₂₋₁
 (u₅ + u₂*P₁₋₁ + 2*u₃*P₂₋₁)*(C₄₋₁ + 2*C₁₋₁*P₁₋₁ + C₂₋₁*P₂₋₁) - (C₅₋₁ + C₂₋₁*P₁₋₁ + 2*C₃₋₁*P₂₋₁)*(u₄ + 2*u₁*P₁₋₁ + u₂*P₂₋₁)
 1 + u₁*P₁₋₂^2 + u₃*P₂₋₂^2 + u₄*P₁₋₂ + u₅*P₂₋₂ + u₂*P₁₋₂*P₂₋₂
 C₆₋₂ + C₁₋₂*P₁₋₂^2 + C₃₋₂*P₂₋₂^2 + C₄₋₂*P₁₋₂ + C₅₋₂*P₂₋₂ + C₂₋₂*P₁₋₂*P₂₋₂
 (u₅ + u₂*P₁₋₂ + 2*u₃*P₂₋₂)*(C₄₋₂ + 2*C₁₋₂*P₁₋₂ + C₂₋₂*P₂₋₂) - (C₅₋₂ + C₂₋₂*P₁₋₂ + 2*C₃₋₂*P₂₋₂)*(u₄ + 2*u₁*P₁₋₂ + u₂*P₂₋₂)
 1 + u₁*P₁₋₃^2 + u₃*P₂₋₃^2 + u₄*P₁₋₃ + u₅*P₂₋₃ + u₂*P₁₋₃*P₂₋₃
 C₆₋₃ + C₁₋₃*P₁₋₃^2 + C₃₋₃*P₂₋₃^2 + C₄₋₃*P₁₋₃ + C₅₋₃*P₂₋₃ + C₂₋₃*P₁₋₃*P₂₋₃
 (u₅ + u₂*P₁₋₃ + 2*u₃*P₂₋₃)*(C₄₋₃

In [24]:
mixed_volume(F)

27072

To solve the system symbolically, you would first eliminate the variables $P_{ij}$.

Then our system would reduce to a system $T$ of five equations in five unknowns of degree 6.

However, each polynomial in the system is dense and has 3210 terms (wrt to $u$) (This makes the system *very expensive* to evaluate).

**There are tradeoffs**

* System $T$ has only mixed volume $6^5=7776$ compared to mixed volume $27072$ for $F$.
* $F$ is substantially cheaper to evaluate and substantially faster for performing parameter homotopy

When you deal with parameterized systems the possibly larger mixed volume is often not a problem since you can use monodromy:

In [ ]:
monodromy_solve(F; target_solutions_count = 3264)


  tracked loops (queued):            397 (368)

  solutions in current (last) loop:  381 (3)

  generated loops (no change):       2 (0)
  generated loops (no change):       2 (0)

## Julia: Use local environments for your projects

Often you need to get back to your projects after a couple months or even years. It would be great if the program you wrote for your project would still work, wouldn't it? Or did you ever experience that things work on your computer but not on you collaborators?

Julia's package manager allows you create *local environments*. There you can list all you package dependencies (in a `Project.toml` file) and it automatically records the exact version used (in a `Manifest.toml`) file.

*Hands on demo*

## HomotopyContinuation.jl: Compiled and interpreted systems

The `solve` and `monodromy_solve` routine HomotopyContinuation.jl have an important option: `compile`.

This topic is a little bit more technical, but it will helper you better understand 

Take our system:

In [3]:
@var x y
F = System([ x^2 - 2*x*y + 4, y^2 - 4])

System of length 2
 2 variables: x, y

 4 - 2*x*y + x^2
 -4 + y^2

We defined our system `F` by using the *symbolic* variables `x` and `y`.
When we pass `F` to `solve` we need to evaluate `F` and its Jacobian many many times. Therefore, we need to be able to do this *fast*. 
We could do:

In [62]:
using BenchmarkTools
pt = [2.0+1im, 4.0+3im]
@btime F($pt)

  6.225 μs (30 allocations: 720 bytes)


2-element Vector{ComplexF64}:
 -2.999999999999999 - 16.0im
                3.0 + 23.999999999999996im

Can we do this any faster?  Yes!

In `HomotopyContinuation.jl` there are actually two ways to do it much faster, using either a `CompiledSystem` or an `InterpretedSystem` (we get to what the difference is in a second):

In [63]:
FC = CompiledSystem(F)

Compiled: System of length 2
 2 variables: x, y

 4 + x*(x - 2*y)
 -4 + y^2

In [65]:
u = zeros(ComplexF64, 2)
@btime evaluate!($u, $FC, $pt)

  5.541 ns (0 allocations: 0 bytes)


2-element Vector{ComplexF64}:
 -3.0 - 16.0im
  3.0 + 24.0im

Let's see how much slower our original approach is:

In [67]:
(@belapsed F($pt)) / (@belapsed evaluate!($u, $FC, $pt))

1167.0456596282258

There is also the `InterpretedSystem` approach

In [68]:
FI = InterpretedSystem(F)

Interpreted: System of length 2
 2 variables: x, y

 4 + x*(x - 2*y)
 -4 + y^2

In [70]:
@btime evaluate!($u, $FI, $pt)

  43.013 ns (0 allocations: 0 bytes)


2-element Vector{ComplexF64}:
 -3.0 - 16.0im
  3.0 + 24.0im

This is still much faster, but a little bit slower than `CompiledSystem`.

The speedup compared to the original way is:

In [74]:
(@belapsed F($pt)) / (@belapsed evaluate!($u, $FI, $pt))

151.67201075217204

### What is happening here?

`CompiledSystem` and `InterpretedSystem` both take our `System` `F` and first transform it into a *straight line program* (SLP).

In [83]:
instructions, out = ModelKit.instruction_list(F.expressions);
instructions

ι_1 = INSTR_MUL(x, y)
ι_2 = INSTR_MULADD(-2, ι_1, 4)
ι_3 = INSTR_SQR(x)
ι_4 = INSTR_ADD(ι_3, ι_2)
ι_5 = INSTR_SQR(y)
ι_6 = INSTR_ADD(ι_5, -4)


Compare this again to `F`

In [84]:
F

System of length 2
 2 variables: x, y

 4 - 2*x*y + x^2
 -4 + y^2

`out` tells us which instructions are the final values of the system entries.

In [79]:
out

2-element Vector{HomotopyContinuation.ModelKit.InstructionRef}:
 ι_4
 ι_6

The difference between `CompiledSystem` and `InterpretedSystem` is how they further process this SLP.

**InterpretedSystem**

This basically takes our SLP (which is just a list of instructions) and iterates over them, stores temporary values etc.

**CompiledSystem**

This actually automatically writes for you a function for your particular system which then gets *compiled*. This compiled function can be heavily optimized and most likely as fast or faster than what you would have implemented manually.

In [89]:
ModelKit._evaluate!_impl(typeof(FC))

quote
    #= /Users/sascha/.julia/packages/HomotopyContinuation/XnHM3/src/model_kit/slp_compiler.jl:86 =#
    begin
        #= /Users/sascha/.julia/packages/HomotopyContinuation/XnHM3/src/model_kit/slp_compiler.jl:27 =# @boundscheck u === nothing || checkbounds(u, 1:2)
        #= /Users/sascha/.julia/packages/HomotopyContinuation/XnHM3/src/model_kit/slp_compiler.jl:29 =# @boundscheck checkbounds(x, 1:2)
        #= /Users/sascha/.julia/packages/HomotopyContinuation/XnHM3/src/model_kit/slp_compiler.jl:30 =# @boundscheck p === nothing || checkbounds(p, 1:0)
    end
    #= /Users/sascha/.julia/packages/HomotopyContinuation/XnHM3/src/model_kit/slp_compiler.jl:87 =#
    #= /Users/sascha/.julia/packages/HomotopyContinuation/XnHM3/src/model_kit/slp_compiler.jl:87 =# @inbounds begin
            #= /Users/sascha/.julia/packages/HomotopyContinuation/XnHM3/src/model_kit/slp_compiler.jl:88 =#
            begin
                ι1 = muladd_instr(-2, x[2], x[1])
                ι2 = muladd_instr(ι1, x

In [99]:
@code_typed evaluate!(u, FC, pt, nothing)

CodeInfo(
1 ──       goto #8 if not $(Expr(:boundscheck))
2 ──       goto #4 if not false
3 ──       nothing::Nothing
4 ┄─ %4  = Core.tuple($(QuoteNode(1:2)))::Core.Const((1:2,))
│    %5  = Base.arraysize(u, 1)::Int64
│    %6  = Base.slt_int(%5, 0)::Bool
│    %7  = Base.ifelse(%6, 0, %5)::Int64
│    %8  = Base.slt_int(2, 1)::Bool
│    %9  = Base.sle_int(1, 1)::Bool
│    %10 = Base.sle_int(1, %7)::Bool
│    %11 = Base.and_int(%9, %10)::Bool
│    %12 = Base.sle_int(1, 2)::Bool
│    %13 = Base.sle_int(2, %7)::Bool
│    %14 = Base.and_int(%12, %13)::Bool
│    %15 = Base.and_int(%11, %14)::Bool
│    %16 = Base.or_int(%8, %15)::Bool
└───       goto #6 if not %16
5 ──       goto #7
6 ──       invoke Base.throw_boundserror(_2::Vector{ComplexF64}, %4::Tuple{UnitRange{Int64}})::Union{}
└───       unreachable
7 ──       nothing::Nothing
8 ┄─       goto #13 if not $(Expr(:boundscheck))
9 ── %23 = Core.tuple($(QuoteNode(1:2)))::Core.Const((1:2,))
│    %24 = Base.arraysize(x, 1)::Int64
│    %25 = Ba

### What are the tradeoffs

**CompiledSystem**:

**+**
* Fast

**-**
* Takes time to compile (doesn't scales so well for large systems)
* Every "new" system has to be compiled


**InterpretedSystem**:

**+**
* No need to compile things again

**-**
* Slower than the compiled approach

Let me illustrate the reocompilation:

In [31]:
rand_poly([x, y], 4)

0.776107180875075 - 1.0320739105085*im + (-0.900096309058035 + 0.331002095610112*im)*x + (-0.226807450520723 + 0.325801962513008*im)*y + (1.55493527026397 - 0.039214892949345*im)*x*y + (-0.380826426470644 + 0.536489041143586*im)*x*y^2 + (-0.656980843146676 + 0.0930402172372995*im)*x*y^3 + (-0.270955911186525 - 0.0689222595566265*im)*x^2*y + (-1.54140686048056 + 0.0658991069839086*im)*x^2*y^2 + (0.28357043932699 - 0.717237366468823*im)*x^3*y + (0.849243014645667 - 0.890222156461549*im)*x^2 + (0.440076780798772 - 0.189911532332772*im)*x^3 + (0.12933892093546 - 0.131721501562058*im)*x^4 + (1.22228539582076 + 0.216040453876466*im)*y^2 + (0.535688087229747 + 0.490157083643858*im)*y^3 + (1.8089864017318 - 1.56680461211209*im)*y^4

In [57]:
G = System([rand_poly([x, y], 4), rand_poly([x, y], 3)])

System of length 2
 2 variables: x, y

 0.738761784399863 - 0.372426920836948*im + (-0.187577145023953 - 0.255128036356298*im)*x + (0.717169726889035 + 0.47416235885744*im)*y + (-0.906677739762466 - 0.44809294081095*im)*x*y + (1.23130971557186 + 1.29123091793069*im)*x*y^2 + (1.21062732119352 + 1.7208985495575*im)*x*y^3 + (0.687697019212547 - 0.568787127648395*im)*x^2*y + (0.440671810862765 + 0.863415263363971*im)*x^2*y^2 + (-0.143531848908182 + 0.00966251167574316*im)*x^3*y + (-0.184875886411293 + 1.05662710412844*im)*x^2 + (0.0626599357250505 - 0.245794588732154*im)*x^3 + (0.480709958018087 + 0.155642166993522*im)*x^4 + (0.356116579918516 + 0.638294204299868*im)*y^2 + (0.0673474546330565 + 0.202745387611684*im)*y^3 + (0.237656672296957 - 1.5807333989468*im)*y^4
 0.0152147182540225 + 0.0951943624682839*im + (-0.302400739156019 + 0.589563090368547*im)*x + (-0.131686068503997 + 0.121945733163676*im)*y + (1.64897218067488 + 0.00757842300588281*im)*x*y + (0.0110537482994193 + 0.56584164668

In [58]:
GC = CompiledSystem(G)
GI = InterpretedSystem(G)

Interpreted: System of length 2
 2 variables: x, y

 0.738761784399863 - 0.372426920836948*im + x*(-0.187577145023953 - 0.255128036356298*im + x*(-0.184875886411293 + 1.05662710412844*im + x*(0.0626599357250505 - 0.245794588732154*im + (0.480709958018087 + 0.155642166993522*im)*x + (-0.143531848908182 + 0.00966251167574316*im)*y) + y*(0.687697019212547 - 0.568787127648395*im + (0.440671810862765 + 0.863415263363971*im)*y)) + y*(-0.906677739762466 - 0.44809294081095*im + y*(1.23130971557186 + 1.29123091793069*im + (1.21062732119352 + 1.7208985495575*im)*y))) + y*(0.717169726889035 + 0.47416235885744*im + y*(0.356116579918516 + 0.638294204299868*im + y*(0.0673474546330565 + 0.202745387611684*im + (0.237656672296957 - 1.5807333989468*im)*y)))
 0.0152147182540225 + 0.0951943624682839*im + x*(-0.302400739156019 + 0.589563090368547*im + x*(0.656731161274784 - 0.897896675986685*im + (-0.529032417738605 + 0.0886676368363951*im)*x + (-0.393456716612199 - 0.35438543904892*im)*y) + y*(1.648972180

In [59]:
@time evaluate!(u, GC, pt)
@time evaluate!(u, GI, pt);

  0.056136 seconds (170.21 k allocations: 10.076 MiB, 99.58% compilation time)
  0.000005 seconds


In [60]:
@btime evaluate!($u, $GC, $pt)
@btime evaluate!($u, $GI, $pt);

  20.770 ns (0 allocations: 0 bytes)
  163.860 ns (0 allocations: 0 bytes)


### What should I take away from this

You most likely never constructed a `CompiledSystem` or `InterpretedSystem` explicitly.
Instead you control this often by setting the `compile` flag in `solve`.

The `compile` option has 3 possible values

* `:none` (or `false`) (will only use `InterpredSystem`)
* `:mixed` (will use the `CompiledSystem` for evaluating $F(x,p)$ and its Jacobian, the intpreted approach for the rest (higher order derivatives))
* `:all` (or `true`) (will only use `CompiledSystem`)

The default value is `:mixed`. Using `:all` can become quickly very expensive.

In [107]:
@time solve(G, compile = :none)

  0.007992 seconds (11.78 k allocations: 667.812 KiB)


Result with 12 solutions
• 12 paths tracked
• 12 non-singular solutions (0 real)
• random_seed: 0x4a1be277
• start_system: :polyhedral


In [109]:
@time solve(G, compile = :mixed)

  0.009016 seconds (15.10 k allocations: 910.648 KiB)


Result with 12 solutions
• 12 paths tracked
• 12 non-singular solutions (0 real)
• random_seed: 0x19b0211b
• start_system: :polyhedral


Instead of setting `compile` for every call, you also set it globally by using `set_default_compile`

In [110]:
?set_default_compile

search: set_default_compile



```
set_default_compile(mode::Symbol)
```

Set the default value for the `compile` flag in [`solve`](@ref) and other functions. Possible values are `:mixed` (default), `:all` and `:none`.


In [111]:
set_default_compile(:none)

:none